In [1]:
import pandas as pd
import psycopg2

# BEFORE RUNNING, MAKE SURE THIS NOTEBOOK IS IN THE SAME DIRECTORY AS ALL THE .CSV FILES IT CAME WITH!

#establishing the connection
conn = psycopg2.connect(
    database='daslemon',
    user='dw_chancewiese',
    password='Spikeball2020',
    host='database-1.czsooswggscz.us-east-2.rds.amazonaws.com',
    port='5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

query = '''CREATE TABLE IF NOT EXISTS daily (
           entryid SERIAL PRIMARY KEY,
           sdate DATE NOT NULL,
           branch CHAR(1),
           regno SMALLINT,
           salestot NUMERIC(7,2),
           txct SMALLINT,
           srcfile VARCHAR(20)
        );'''

cursor.execute(query)
conn.commit()

query = '''CREATE TABLE IF NOT EXISTS ingest (
           filename VARCHAR(20) PRIMARY KEY,
           starttime TIMESTAMP,
           endtime TIMESTAMP
        );'''

cursor.execute(query)
conn.commit()

In [2]:
filenames = ['a20231011.csv','b20231011.csv','c20231011.csv','d20231011.csv',
            'e20231011.csv','a20231012.csv','b20231012.csv','c20231012.csv',
            'd20231012.csv','e20231012.csv','d20231013.csv','e20231013.csv',]

for f in filenames:

    df = pd.read_csv(f)
    
    branch = f[0]
    date = f[1:5] + '-' + f[5:7] + '-' + f[7:9]
    
    
    for x in df.index:
        query = '''INSERT INTO daily (sdate, branch, regno, salestot, txct, srcfile)
                VALUES ('%s', '%s', %s, %s, %s, '%s')''' % (date, branch, df['register'].loc[x], 
                                                    df['totalsales'].loc[x],
                                                    df['transactions'].loc[x], f)
        print(query)

        cursor.execute(query)
        conn.commit()

    query = '''INSERT INTO ingest (filename) VALUES ('%s')''' % (f)
    print(query)
    
    cursor.execute(query)
    conn.commit()

INSERT INTO daily (sdate, branch, regno, salestot, txct, srcfile)
                VALUES ('2023-10-11', 'a', 1, 272.08, 34, 'a20231011.csv')
INSERT INTO daily (sdate, branch, regno, salestot, txct, srcfile)
                VALUES ('2023-10-11', 'a', 2, 247.4, 30, 'a20231011.csv')
INSERT INTO ingest (filename) VALUES ('a20231011.csv')
INSERT INTO daily (sdate, branch, regno, salestot, txct, srcfile)
                VALUES ('2023-10-11', 'b', 1, 249.65, 30, 'b20231011.csv')
INSERT INTO daily (sdate, branch, regno, salestot, txct, srcfile)
                VALUES ('2023-10-11', 'b', 2, 232.25, 28, 'b20231011.csv')
INSERT INTO ingest (filename) VALUES ('b20231011.csv')
INSERT INTO daily (sdate, branch, regno, salestot, txct, srcfile)
                VALUES ('2023-10-11', 'c', 1, 232.81, 28, 'c20231011.csv')
INSERT INTO daily (sdate, branch, regno, salestot, txct, srcfile)
                VALUES ('2023-10-11', 'c', 2, 260.02, 31, 'c20231011.csv')
INSERT INTO ingest (filename) VALUES ('c20231

In [3]:
query = '''SELECT * FROM daily;'''

df = pd.read_sql(query,conn)
print(df.head(15))

    entryid       sdate branch  regno  salestot  txct        srcfile
0         1  2023-10-11      a      1    272.08    34  a20231011.csv
1         2  2023-10-11      a      2    247.40    30  a20231011.csv
2         3  2023-10-11      b      1    249.65    30  b20231011.csv
3         4  2023-10-11      b      2    232.25    28  b20231011.csv
4         5  2023-10-11      c      1    232.81    28  c20231011.csv
5         6  2023-10-11      c      2    260.02    31  c20231011.csv
6         7  2023-10-11      d      1    258.06    31  d20231011.csv
7         8  2023-10-11      d      2    217.11    26  d20231011.csv
8         9  2023-10-11      e      1    274.89    34  e20231011.csv
9        10  2023-10-11      e      2    224.68    28  e20231011.csv
10       11  2023-10-12      a      1    484.00    60  a20231012.csv
11       12  2023-10-12      a      2    499.95    62  a20231012.csv
12       13  2023-10-12      b      1    583.00    74  b20231012.csv
13       14  2023-10-12      b    

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_55503/2568083612.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,conn)


In [11]:
conn.close()